In [1]:
import pandas as pd

In [3]:
import tensorflow as tf

In [29]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [41]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [49]:
from tensorflow.keras.models import Sequential

In [51]:
from tensorflow.keras.layers import Dense, Input

In [131]:
from tensorflow.keras.optimizers import SGD

In [5]:
df = pd.read_csv("../../../dados/nlp/imdb-reviews-pt-br.csv", encoding="utf-8")

In [213]:
df

,id,text_en,text_pt,sentiment,cat_sentiment,texto_limpo
0,1,Once again Mr. Costner has dragged out a movie...,"Mais uma vez, o Sr. Costner arrumou um filme p...",neg,0,"mais uma vez, o sr. costner arrumou um filme p..."
1,2,This is an example of why the majority of acti...,Este é um exemplo do motivo pelo qual a maiori...,neg,0,este é um exemplo do motivo pelo qual a maiori...
2,3,"First of all I hate those moronic rappers, who...","Primeiro de tudo eu odeio esses raps imbecis, ...",neg,0,"primeiro de tudo eu odeio esses raps imbecis, ..."
3,4,Not even the Beatles could write songs everyon...,Nem mesmo os Beatles puderam escrever músicas ...,neg,0,nem mesmo os beatles puderam escrever músicas ...
4,5,Brass pictures movies is not a fitting word fo...,Filmes de fotos de latão não é uma palavra apr...,neg,0,filmes de fotos de latão não é uma palavra apr...
...,...,...,...,...,...,...
49454,49456,"Seeing as the vote average was pretty low, and...","Como a média de votos era muito baixa, e o fat...",pos,1,"como a média de votos era muito baixa, e o fat..."
49455,49457,"The plot had some wretched, unbelievable twist...",O enredo teve algumas reviravoltas infelizes e...,pos,1,o enredo teve algumas reviravoltas infelizes e...
49456,49458,I am amazed at how this movieand most others h...,Estou espantado com a forma como este filme e ...,pos,1,estou espantado com a forma como este filme e ...
49457,49459,A Christmas Together actually came before my t...,A Christmas Together realmente veio antes do m...,pos,1,a christmas together realmente veio antes do m...


In [215]:
df = df.assign(cat_sentiment = [1 if i == "pos" else 0  for i in df["sentiment"]])
df["texto_limpo"] = df["text_pt"].str.lower()

In [225]:
tokenizer = Tokenizer( num_words = 1000 )

In [227]:
tokenizer.fit_on_texts( df["texto_limpo"] )

In [228]:
sequences = tokenizer.texts_to_sequences(df["texto_limpo"])

In [229]:
num_words = len(tokenizer.word_index) + 1
num_words

130862

In [230]:
maior = 0
for sequence in sequences: 
    if len(sequence) > maior:
        maior = len(sequence)
print("Maior sequencia: ", maior)

Maior sequencia:  760


In [231]:
sequences_padded = pad_sequences(sequences, padding="post")

In [232]:
sequences_padded.shape

(49459, 760)

In [233]:
X = sequences_padded
Y = df["cat_sentiment"]

In [234]:
X.shape

(49459, 760)

In [235]:
Y.shape

(49459,)

In [236]:
modelo = Sequential()
modelo.add( Input(shape=(maior,), dtype="int32", name="Entrada") ) 
# modelo.add( Dense(units=200, activation="relu", name="Camada_1") )
modelo.add( Dense(units=100, activation="relu", name="Camada_2") )
# modelo.add( Dense(units=10, activation="relu", name="Camada_3") )
modelo.add( Dense(units=1, activation="sigmoid", name="Saida") )
modelo.summary()

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ Camada_2 (Dense)                     │ (None, 100)                 │          76,100 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ Saida (Dense)                        │ (None, 1)                   │             101 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 76,201 (297.66 KB)

 Trainable params: 76,201 (297.66 KB)

 Non-trainable params: 0 (0.00 B)

In [237]:
opt = SGD( learning_rate = 0.1 )
modelo.compile( optimizer = opt, loss = "binary_crossentropy", metrics=["accuracy"] )

In [238]:
historico = modelo.fit(X, Y, epochs=10, batch_size=64, verbose=1)

Epoch 1/10
773/773 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.5011 - loss: 49218.6289
Epoch 2/10
773/773 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.4961 - loss: 3.4862
Epoch 3/10
773/773 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.5013 - loss: 0.6931
Epoch 4/10
773/773 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.5025 - loss: 0.6932
Epoch 5/10
773/773 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.4993 - loss: 0.6932
Epoch 6/10
773/773 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.4962 - loss: 0.6932
Epoch 7/10
773/773 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.4981 - loss: 0.6931
Epoch 8/10
773/773 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.5020 - loss: 0.6932
Epoch 9/10
773/773 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.4990 - loss: 0.6932
Epoch 10/10
773/773 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.4989 - loss: 0.6932


In [239]:
historico

In [240]:
historico.history

{'accuracy': [0.5007379651069641,
  0.4984532594680786,
  0.4993833303451538,
  0.5006975531578064,
  0.4982510805130005,
  0.4977456033229828,
  0.49835216999053955,
  0.5013040900230408,
  0.49835216999053955,
  0.5017085075378418],
 'loss': [8601.1298828125,
  1.9016246795654297,
  0.6931979060173035,
  0.6932059526443481,
  0.6932259202003479,
  0.6932253241539001,
  0.6932005286216736,
  0.6931927800178528,
  0.6932290196418762,
  0.6932061314582825]}